# Detection of Toxic Comments

Kaggle competition: [Toxic Comment Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge)

In [1]:
seed = 333

## Import

In [2]:
import csv
import string
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.


## Load Data

In [5]:
def read_comments(filename = ''):
    comments = []
    labels = []
    ids = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        header = True
        for row in csvReader:
            if header:
                header = False
                continue
            ids.append(row[0])
            s = row[1].translate(string.punctuation)
            comments.append(s)          
            if len(row) > 2:
                classes = []
                for i in range(2,8):
                    classes.append(row[i])
                labels.append(classes)

    ids = np.asarray(ids)
    X = np.asarray(comments)
    Y = np.asarray(labels, dtype=int)

    return ids, X, Y

In [6]:
id_train, X_train, Y_train = read_comments('./data/train.csv')
id_test, X_test, _ = read_comments('./data/test.csv')
labels = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
max_comment = len(max(X_train, key=len).split())

## Split Data

In [7]:
kfold = KFold(n_splits=5,random_state=seed)
# folds = list()
# for train_index, dev_index in kfold.split(X_train):
#     folds.append({
#         'train_index' : train_index,
#         'dev_index' : dev_index,     
#     })

## Load Embeddings

In [11]:
def read_embeddings(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [12]:
word_to_index, index_to_word, word_to_vec = read_embeddings('./glove/glove.twitter.27B.25d.txt')

In [13]:
for r in X_train:
    words = r.split()
    for w in words:
        if w not in word_to_index:
            if 'fuck' in w:
                print(w)



fistfuckee.
fuck.
fuck.++You
motherfucker!!!
++fuck
fuck.
Assfuckers
muthafucker.
""fucking
++Muthafucka
fuck.
dumbfucks.
muthafucka!
mothafucka!
clusterfucked
""clusterfucked
motherfucker.
""fucking
fuck...Before
http://gawker.com/5636765/facebook-ceo-admits-to-calling-users-dumb-fucks+
http://www.theweek.co.uk/technology/14625/are-users-%E2%80%98dumb-fucks%E2%80%99-trusting-data-facebook+
http://tdh.me/zuckerberg-called-early-facebook-users-dumb-fucks-so-what/+
http://anphicle.com/en/they-trust-me-dumb-fucks-facebook-ceo-mark-zuckerberg/+
fucks"".
fucks.++In
Corpsefucking
Corpsefucking
Corpsefucking
Corpsefucking
changes++fuck
motherfucker""
dumass.++fuck
""fucking
fuckkk
""mindfuck""
fuck's
Mindfuck
fuck?
fuckface.
fuck.
fuckin'
motherfucker,
bitch.++fuck
lord/fuck
fuckersItalic
dumbfuck,
fuck,
fuck????
fucking-ass
++fuck
Merchan-fucking-dise
T-fucking-800.
fuck...damn
""fuck""
""fuck,""
""fuck""
""fuck""
""fucking"")
""fuck""
""fuck""
""fuck""
""fuck""
""fuck""
fuck.
'fuck'
""fuck.

## Baseline model

This is naive model based on averaging all word embeddings in a comment.

In [14]:
def average_comment(comment, word_to_vec):
    """
    Converts a comment into a list of words. 
    Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    comment -- string
    word_to_vec_map -- dictionary mapping every word in a vocabulary 
    into its n-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (n,)
    """
    words = comment[0].lower().split()
    avg = np.zeros(shape=(len(word_to_vec['apple']), ))
    for w in words:
        if w in word_to_vec:
            avg += word_to_vec[w]
    avg = avg / len(words)
    return avg

In [15]:
averages = list()
for c in X_train:
    averages.append(average_comment(c, word_to_vec))
averages = np.asarray(averages)

In [16]:
from sklearn.metrics import log_loss
from keras.backend import int_shape
import keras.backend as K
import tensorflow as tf
_EPSILON = K.epsilon()

def custom_loss(y_true, y_pred):
    losses = 0.0
    y_pred = K.clip(y_pred, _EPSILON, 1.0-_EPSILON)
    num_classes = int_shape(y_pred)[1]
    num_samples = int_shape(y_pred)[0]
    for i in range(num_classes):
        #losses += log_loss(y_true[:, i], y_pred[:, i])
        losses += -(
            y_true[:, i]*K.log(y_pred[:, i]) - (1-y_true[:, i])*K.log(1-y_pred[:, i])
        )

    return tf.convert_to_tensor(losses, dtype='float32')
        
#     return np.mean([log_loss(y_true[:, i], y_pred[:, i]) 
#                     for i in range(int_shape(y_pred)[1])])

def baseline_model(num_dim=25, num_labels=6):
    # create model
    model = Sequential()
    model.add(Dense(num_labels, input_shape=(num_dim,), kernel_initializer='normal', activation='sigmoid'))
    model.compile(
        loss='binary_crossentropy', 
        #loss=custom_loss,
        optimizer='adam', metrics=['accuracy'])
    return model

In [17]:
model = baseline_model()

In [18]:
model.fit(averages, Y_train, epochs=4, verbose=1)

Epoch 1/4
159571/159571 [==============================] - 13s - loss: 0.1504 - acc: 0.9615    
Epoch 2/4
159571/159571 [==============================] - 12s - loss: 0.1351 - acc: 0.9633    
Epoch 3/4
159571/159571 [==============================] - 12s - loss: 0.1341 - acc: 0.9633    
Epoch 4/4
159571/159571 [==============================] - 12s - loss: 0.1336 - acc: 0.9633    


In [19]:
model.predict(averages[3003:3005])

array([[ 0.07618176,  0.00648048,  0.04228055,  0.0020038 ,  0.03684543,
         0.00833923],
       [ 0.12135574,  0.01604569,  0.07556893,  0.00372793,  0.06651573,
         0.01333715]], dtype=float32)

In [20]:
Y_train[3003:3005]

array([[0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0]])

In [21]:
np.random.seed(seed)
estimator_baseline = KerasRegressor(build_fn=baseline_model, 
                                    nb_epoch=1, batch_size=5, verbose=1)

In [ ]:
results = cross_val_score(estimator_baseline, averages, Y_train, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/10
127656/127656 [==============================] - 64s - loss: 0.1387 - acc: 0.9630    
Epoch 2/10
 98175/127656 [======================>.......] - ETA: 14s - loss: 0.1329 - acc: 0.9637

In [24]:
model.predict(np.asarray([average_comment("You", word_to_vec)]))

array([[ 0.2448447 ,  0.03423478,  0.1390679 ,  0.00968862,  0.1592744 ,
         0.02638858]], dtype=float32)

In [25]:
Y_train[0]

array([0, 0, 0, 0, 0, 0])